In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
import numpy
import lightgbm
from xgboost import XGBClassifier
from sklearn.metrics import balanced_accuracy_score

In [48]:
dfB = pd.read_parquet('benign_2312_HTML.parquet')
dfB

,domain_name,dns_has_dnskey,dns_A_count,dns_AAAA_count,dns_MX_count,dns_NS_count,dns_TXT_count,dns_SOA_count,dns_CNAME_count,dns_zone_level,...,html_num_of_form_hash,html_num_of_form_js,html_malicious_form,html_most_common,html_num_of_css_internal,html_num_of_css_external,html_num_of_anchors_to_content,html_num_of_anchors_to_void,html_num_of_blank_spaces,html_blocked_keywords_label
0,0000100m0030.han2.savba.sk,0.0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,000000.tpn01.edugroup.at,0.0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,30,0
2,000000.pn01.edugroup.at,0.0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,422,0
3,42.svetacdn.in,0.0,2,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,000111.tpn01.edugroup.at,0.0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171217,careerengagement.institute,0.0,1,0,1,2,3,1,0,0,...,2,0,1,0,0,0,0,0,33,0
171218,beaumarissc.vic.edu.au,0.0,1,0,1,3,5,1,0,0,...,0,0,1,0,0,0,0,0,794,0
171219,taker.rest,0.0,2,2,0,2,1,1,0,0,...,0,0,0,0,0,0,0,0,102,0
171220,emberger.or.at,0.0,1,0,1,2,2,1,0,0,...,0,0,0,0,0,0,0,0,351,0


In [49]:
dfB = pd.read_parquet('benign_2312_HTML.parquet')


filtered_dfB  = dfB.filter(like='html', axis=1)

filtered_dfB['label'] = 'benign'
# Display the filtered DataFrame
print(filtered_dfB['label'])

0         benign
1         benign
2         benign
3         benign
4         benign
           ...  
171217    benign
171218    benign
171219    benign
171220    benign
171221    benign
Name: label, Length: 171222, dtype: object


/tmp/ipykernel_964/4151630429.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_dfB['label'] = 'benign'


In [50]:
dfM = pd.read_parquet('malware_2406_strict_HTML.parquet')


filtered_dfM = dfM.filter(like='html', axis=1)



filtered_dfM['label'] = 'malware'
# Display the filtered DataFrame
print(filtered_dfM)

       html_create_element  html_write  html_char_code_at  html_concat  \
0                        0           0                  0            0   
1                        0           0                  0            0   
2                        1           0                  0            0   
3                        0           0                  0            0   
4                        2           0                  0            0   
...                    ...         ...                ...          ...   
22964                    0           0                  0            0   
22965                    0           0                  0            0   
22966                    2           0                  0            0   
22967                    2           0                  0            0   
22968                    0           0                  0            0   

       html_escape  html_eval  html_exec  html_from_char_code  html_link  \
0                0          0      

/tmp/ipykernel_964/2657990986.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_dfM['label'] = 'malware'


In [52]:
filtered_dfB.dropna(inplace=True)
filtered_dfM.dropna(inplace=True)
filtered_dfM["html_external_href_ratio"] = filtered_dfM["html_external_href_ratio"].astype(bool)
filtered_dfM["html_external_href_ratio"] = filtered_dfM["html_external_href_ratio"].astype(int)
filtered_dfM["html_internal_href_ratio"] = filtered_dfM["html_internal_href_ratio"].astype(bool)
filtered_dfM["html_internal_href_ratio"] = filtered_dfM["html_internal_href_ratio"].astype(int)

combined_df = pd.concat([filtered_dfB, filtered_dfM], axis=0)
print(len(combined_df))


194191


/tmp/ipykernel_964/83181100.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_dfB.dropna(inplace=True)
/tmp/ipykernel_964/83181100.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_dfM.dropna(inplace=True)
/tmp/ipykernel_964/83181100.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_dfM["html_external_href_ratio"] = filtered_dfM["html_external_hre

In [53]:
combined_df

,html_create_element,html_write,html_char_code_at,html_concat,html_escape,html_eval,html_exec,html_from_char_code,html_link,html_parse_int,...,html_num_of_form_js,html_malicious_form,html_most_common,html_num_of_css_internal,html_num_of_css_external,html_num_of_anchors_to_content,html_num_of_anchors_to_void,html_num_of_blank_spaces,html_blocked_keywords_label,label
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0.000000,0,0,0,0,0,0,benign
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0.000000,0,0,0,0,30,0,benign
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0.000000,0,0,0,0,422,0,benign
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0.000000,0,0,0,0,1,0,benign
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0.000000,0,0,0,0,5,0,benign
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22964,0,0,0,0,0,0,0,0,0,0,...,0,1,0.333333,0,0,0,0,419,0,malware
22965,0,0,0,0,0,0,0,0,0,0,...,0,1,0.333333,0,0,0,0,419,0,malware
22966,2,0,0,0,0,0,0,0,0,0,...,0,0,0.182825,0,0,0,0,4753,0,malware
22967,2,0,0,0,0,0,0,0,0,0,...,0,1,0.142857,0,0,0,0,169,0,malware


In [54]:
bool_cols = combined_df.select_dtypes(include='bool').columns
print(bool_cols)
combined_df[bool_cols] = combined_df[bool_cols].astype(int)

pdaFeatures = combined_df.loc[:, combined_df.columns != "label"] # get features
pdaLabel = pd.DataFrame() 
pdaLabel["label"] = combined_df["label"] # get labels

print(len(pdaFeatures))

Index(['html_no_hrefs'], dtype='object')
194191


In [55]:
dX_train, X_test, y_train, y_test = train_test_split(pdaFeatures, pdaLabel, test_size=0.20, random_state=42) # split data into training and testing data

In [56]:
lightGBMClas = lightgbm.LGBMClassifier(
    objective="binary",
    is_unbalance=True,
    max_depth=13,
    num_leaves=261,
    n_estimators=1498,
    lambda_l1=0.0017993225169621843,
    lambda_l2=0.0006301826170072739,
    bagging_fraction=0.16163071559346373,
    bagging_freq=0,
    max_bin=800,
    learning_rate=0.05437419665350978,
    min_child_samples=3,
    boosting_type="gbdt",
    min_split_gain=0.000010284197711702238,
    pos_bagging_fraction=0.8451744872078454,
    neg_bagging_fraction=0.8714695064033464,
    colsample_bytree=0.42342506718022915,
    verbosity=-1,
    seed = 500
) # set hyperparameters
npLabel: numpy.ndarray = y_train.to_numpy() 
lightGBMClas.fit(dX_train, npLabel.ravel()) # fit model
prediction = lightGBMClas.predict(X_test)
print("F1 score: ", f1_score(y_test, prediction,pos_label='malware'))
print("Accuracy: ", accuracy_score(y_test, prediction))
print("Balanced accuracy: ", balanced_accuracy_score(y_test, prediction))

F1 score:  0.8907011226435078
Accuracy:  0.9734287700507223
Balanced accuracy:  0.9532963441374027


In [47]:
import joblib
joblib.dump(lightGBMClas, "models/malware_html_lgbm_model_updated.joblib")

['malware_html_lgbm_model_updated.joblib']